In [2]:
#NOTE: if you ever get issues with the map not drawing your data correctly, remove the layer your missing from the map and hit control + Z
arcpy.env.workspace = r"C:\Local_GIS_Scripting\Water Resources Sub Hub Help\Master_WR_Content.gdb"

input_polygons = "Polygons_Intersect_Dissolve" # Watershed Polygons
input_master_dataset = "State" # Set either to metro or state
input_ID_dissolve_field = "FID_Polygons" # If your data is coming over from a kml leave this alone
input_project_name = "Hwy_71"

master_dataset = f'{input_master_dataset}_CN_Polys'
arcpy.management.SelectLayerByLocation(master_dataset, "INTERSECT", input_polygons, selection_type = "NEW_SELECTION")

intersect_output = fr"CN_Intersect_{input_project_name}"
arcpy.analysis.Intersect(f"{input_polygons} #;{master_dataset} #", intersect_output)

# To get the labels, turn on labels, then click labeling properties and replace the expression text with this: $feature.hydgrpdcd + ", " + $feature.Class + ", " + $feature.Value
# After running this block feel free to go in and manually edit some of the curve values you may find from the output. Make sure to save the output after making changes in the edit tab.

<Result 'C:\\Local_GIS_Scripting\\Water Resources Sub Hub Help\\Master_WR_Content.gdb\\CN_Intersect_Hwy_71'>

In [3]:
# Output feature classes
dissolve_output = fr"CN_Dissolved_{input_project_name}"

# Step 2: Add Field 'CN_x_Area' (double) and calculate product of Shape_Area and CN field
arcpy.management.AddField(intersect_output, "CN_x_Area", "DOUBLE")

# Calculate the CN_x_Area field as Shape_Area * CN
with arcpy.da.UpdateCursor(intersect_output, ["SHAPE@AREA", "Value", "CN_x_Area"]) as cursor:
    for row in cursor:
        row[2] = row[0] * row[1]  # CN_x_Area = Shape_Area * CN
        cursor.updateRow(row)

# Step 3: Dissolve on 'FID_Polygons' and sum 'CN_x_Area'
arcpy.management.Dissolve(intersect_output, dissolve_output, input_ID_dissolve_field, [["CN_x_Area", "SUM"]])

# Step 4: Add 'CN_Weighted' field (double)
arcpy.management.AddField(dissolve_output, "CN_Weighted", "DOUBLE")

# Calculate CN_Weighted = summed CN_x_Area / new Shape_Area
with arcpy.da.UpdateCursor(dissolve_output, ["SUM_CN_x_Area", "SHAPE@AREA", "CN_Weighted"]) as cursor:
    for row in cursor:
        row[2] = row[0] / row[1]  # CN_Weighted = SUM(CN_x_Area) / new Shape_Area
        cursor.updateRow(row)